In [19]:
#dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from datetime import datetime
import time

import sys
sys.path.append("..")
from config import weather_api_key

## Deliverable 1 - Gathering Data

In [20]:
#make random latlong pairs
#setting random seed in case I need to come back to this
np.random.seed(1979)

lats = np.random.uniform(-90, 90, size = 2000)
longs = np.random.uniform(-180, 180, size = 2000)

lat_longs = zip(lats, longs)

coordinates = list(lat_longs)

In [21]:
#make list of cities from lat logns
cities = []

for coordinate in coordinates:
    #identify the nearest city
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #add it to the list if it's not already there
    if city not in cities:
        cities.append(city)

#check cities size
print(len(cities))

768


### Get weather for those cities

In [22]:
# list to hold weather data
city_data = []
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

# print beggining of logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

#set up counters
record_count = 1
set_count = 1

for i, city in enumerate(cities):

    #Group cities in sets of 50 to avoid timing out
    if (i % 50 == 0) and (i >= 50):
        set_count +=1
        record_count = 1
        time.sleep(60)

    #log the url, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #build city url note: replace is taking spaces in names and replacing them with + to not just get data for first part of name
    city_url = url + "&q=" + city

    #run an api request on city
    try:
        #get the json data
        city_weather = requests.get(city_url).json()

        #parse the data
        city_lat = city_weather["coord"]["lat"]
        city_long = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_winds = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

        #convert date
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime("%Y-%m-%d %H:%M:%S")

        #add city data to the list
        city_data.append({"City": city.title(),
                        "Lat": city_lat,
                        "Long": city_long,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_winds,
                        "Country": city_country,
                        "Date": city_date,
                        "Current Description": city_description})
    
    #skip the city if it isn't found
    except:
        print(f"City not found. Skipping...")
        pass 

    #add 1 to record count
    record_count += 1

#note that data loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | berlevag
Processing Record 2 of Set 1 | wuchang
Processing Record 3 of Set 1 | makakilo city
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | bethel
Processing Record 6 of Set 1 | umzimvubu
City not found. Skipping...
Processing Record 7 of Set 1 | uberlandia
Processing Record 8 of Set 1 | nemuro
Processing Record 9 of Set 1 | lingao
Processing Record 10 of Set 1 | umm kaddadah
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | tuktoyaktuk
Processing Record 13 of Set 1 | new norfolk
Processing Record 14 of Set 1 | tazmalt
Processing Record 15 of Set 1 | ilulissat
Processing Record 16 of Set 1 | kruisfontein
Processing Record 17 of Set 1 | royan
Processing Record 18 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 19 of Set 1 | puerto ayora
Processing Record 20 of Set 1 | sitka
Processing Record 21 of Set 1 | qaanaaq
Processing Recor

In [24]:
city_data_df = pd.DataFrame(city_data)
col_order = ["City", "Country", "Date", "Lat", "Long", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[col_order]
city_data_df.head(10)

,City,Country,Date,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Berlevag,NO,2022-07-20 20:38:43,70.8578,29.0864,55.20,94,100,2.30,overcast clouds
1,Wuchang,CN,2022-07-20 20:38:43,44.9143,127.1500,67.19,94,97,3.87,overcast clouds
2,Makakilo City,US,2022-07-20 20:38:43,21.3469,-158.0858,80.69,69,40,13.80,scattered clouds
3,Ushuaia,AR,2022-07-20 20:38:44,-54.8000,-68.3000,33.42,93,100,21.85,snow
4,Bethel,US,2022-07-20 20:35:49,41.3712,-73.4140,98.02,50,20,8.05,few clouds
5,Uberlandia,BR,2022-07-20 20:38:45,-18.9186,-48.2772,79.81,27,0,10.36,clear sky
6,Nemuro,JP,2022-07-20 20:38:45,43.3236,145.5750,57.56,91,64,8.77,broken clouds
7,Lingao,CN,2022-07-20 20:38:46,19.9103,109.6859,79.93,87,1,13.78,clear sky
8,Umm Kaddadah,SD,2022-07-20 20:38:46,13.6017,26.6876,85.57,46,27,8.70,light rain
9,Cape Town,ZA,2022-07-20 20:38:14,-33.9258,18.4232,53.11,65,0,3.44,clear sky


In [25]:
#save the df into a csv

city_data_df.to_csv("WeatherPy_Database.csv", index_label="City_ID")